# Dependent Field Validations

So far we have seen validators that act upon some value being deserialized, totally unaware of other fields on the model.

But, when deserializing data, there might be fields, defined before the current one in the field definition order in the model class, that were already deserialized.

Pydantic has a way for a custom validator to receive information about other fields.

The catch is that it can only know about fields that have already been validated, and only those fields that passed validation.

The way this works is that custom validators can receive an extra argument, typed as `ValidationInfo`.

Let's see a example of this:

In [1]:
from pydantic import BaseModel, field_validator, ValidationError, ValidationInfo

In [2]:
class Model(BaseModel):
    field_1: int
    field_2: list[int]
    field_3: str
    field_4: list[str]

    @field_validator("field_3")
    @classmethod
    def validator(cls, value: str, validated_values: ValidationInfo):
        print(f"{value=}")
        print(f"{validated_values=}")
        return value

In [3]:
Model(field_1=100, field_2=[1, 2, 3], field_3="python", field_4=["a", "b"])

value='python'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 100, 'field_2': [1, 2, 3]}, field_name='field_3')


Model(field_1=100, field_2=[1, 2, 3], field_3='python', field_4=['a', 'b'])

As you can see, that `ValidationInfo` object, has an attribute named `data` - it is a dictionary that contains the validated fields that were defined **before** `field_3` in this model.

Recall that Pydantic will keep validating fields even if one fails validation. This means that when you receive that `ValidationInfo` object, it may not contain values for all the previous fields - it only contains values that actually passed validation.

In [4]:
try:
    Model(field_1=100, field_2=["a", "b"], field_3="python", field_4=["a", "b"])
except ValidationError as ex:
    print(ex)

value='python'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 100}, field_name='field_3')
2 validation errors for Model
field_2.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing
field_2.1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='b', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing


As you can see, `field_2` was not included in the `ValidationInfo` object, since that value failed to pass validation.

So, make sure you take that possibility into account.

Let's see a typical application of this.

Suppose we have a model with a start and end datetime - our validation needs to ensure that the end date is not earlier than the start date, so let's implement that.

We'll bring back the validators we had when dealing with datetimes:

In [5]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse
from pydantic import AfterValidator, BeforeValidator

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [6]:
class Model(BaseModel):
    start_dt: DateTimeUTC
    end_dt: DateTimeUTC

    @field_validator("end_dt")
    @classmethod
    def validate_end_after_start_dt(cls, value: datetime, values: ValidationInfo):
        data = values.data
        if "start_dt" in data:
            if value <= data["start_dt"]:
                raise ValueError("end_dt must come after start_dt")
        # if start_dt failed validation, there's not much we can check here. 
        #    So just return value as-is
        return value  

In [7]:
Model(start_dt="2020/1/1", end_dt="2020/12/31")

Model(start_dt=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>), end_dt=datetime.datetime(2020, 12, 31, 0, 0, tzinfo=<UTC>))

In [8]:
try:
    Model(start_dt="2020/1/1", end_dt="2012/12/31")
except ValidationError as ex:
    print(ex)

1 validation error for Model
end_dt
  Value error, end_dt must come after start_dt [type=value_error, input_value='2012/12/31', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error
